In [1]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import pytesseract
import base64
import pandas as pd
import time

# Đặt đường dẫn đến tesseract executable trên Windows
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Hàm để chuyển đổi chuỗi base64 thành hình ảnh
def base64_to_image(base64_str):
    image_data = base64.b64decode(base64_str.split(',')[1])
    return Image.open(BytesIO(image_data))

# Hàm để lấy dữ liệu từ trang công ty
def fetch_company_data(company_link):
    start_time = time.time()
    
    try:
        response = requests.get(company_link, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Failed to retrieve company page {company_link}: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    jumbotron_div = soup.find('div', class_='jumbotron')
    if jumbotron_div is None:
        print(f'Jumbotron div not found on page {company_link}')
        return None

    company_name = jumbotron_div.find('h4').text.strip() if jumbotron_div.find('h4') else 'N/A'

    # Xử lý mã số thuế từ ảnh base64
    tax_code_img_base64 = jumbotron_div.find_all('img')[0]['src'] if len(jumbotron_div.find_all('img')) > 0 else None
    if tax_code_img_base64:
        tax_code_img = base64_to_image(tax_code_img_base64)
        try:
            tax_code_text = pytesseract.image_to_string(tax_code_img, config='--psm 6').strip()
        except Exception as e:
            tax_code_text = 'Error'
    else:
        tax_code_text = 'N/A'

    # Xử lý số điện thoại từ ảnh base64
    phone_img_base64 = jumbotron_div.find_all('img')[1]['src'] if len(jumbotron_div.find_all('img')) > 1 else None
    if phone_img_base64:
        phone_img = base64_to_image(phone_img_base64)
        try:
            phone_text = pytesseract.image_to_string(phone_img, config='--psm 6').strip()
        except Exception as e:
            phone_text = 'Error'
    else:
        phone_text = 'N/A'

    address = jumbotron_div.find(text='Địa chỉ:').next.strip() if jumbotron_div.find(text='Địa chỉ:') else 'N/A'
    legal_representative = jumbotron_div.find(text='Đại diện pháp luật:').next.strip() if jumbotron_div.find(text='Đại diện pháp luật:') else 'N/A'
    status = jumbotron_div.find(text='Trạng thái:').next.strip() if jumbotron_div.find(text='Trạng thái:') else 'N/A'
    
    end_time = time.time()
    print(f"Time taken to fetch and process {company_link}: {end_time - start_time} seconds")

    return {
        'Company Name': f'=HYPERLINK("{company_link}", "{company_name}")',
        'Tax Code': tax_code_text,
        'Phone': phone_text,
        'Legal Representative': legal_representative,
        'Address': address,
        'Status': status
    }

# Hàm để lấy tất cả liên kết công ty từ một trang
def fetch_company_links_from_page(page_number):
    url = f'https://www.tratencongty.com/?page={page_number}'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        company_links = []
        for link in soup.find_all('a', href=True):
            if 'company' in link['href']:
                full_link = link['href'] if link['href'].startswith('http') else 'https://www.tratencongty.com' + link['href']
                company_links.append(full_link)
        return company_links
    else:
        print(f'Failed to retrieve page {page_number}')
        return []

# Hàm để lấy dữ liệu từ tất cả các trang
def fetch_data_from_all_pages(start_page, end_page):
    all_data = []
    for page_number in range(start_page, end_page + 1):
        print(f"Fetching page {page_number}...")
        company_links = fetch_company_links_from_page(page_number)
        for link in company_links:
            company_data = fetch_company_data(link)
            if company_data:
                all_data.append(company_data)
        time.sleep(2)  # Delay between requests to avoid overwhelming the server

    # Chuyển dữ liệu sang DataFrame và xuất ra file Excel
    df = pd.DataFrame(all_data)
    df.to_excel('company_data_all_pages1.xlsx', index=False, engine='openpyxl')
    print('Data has been exported to company_data_all_pages.xlsx')

# Gọi hàm để lấy dữ liệu từ tất cả các trang từ 1 đến 5 (hoặc theo phạm vi trang bạn cần)
fetch_data_from_all_pages(2, 2)


Fetching page 2...


C:\Users\Admin\AppData\Local\Temp\ipykernel_20352\4070490340.py:59: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  address = jumbotron_div.find(text='Địa chỉ:').next.strip() if jumbotron_div.find(text='Địa chỉ:') else 'N/A'
C:\Users\Admin\AppData\Local\Temp\ipykernel_20352\4070490340.py:60: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  legal_representative = jumbotron_div.find(text='Đại diện pháp luật:').next.strip() if jumbotron_div.find(text='Đại diện pháp luật:') else 'N/A'
C:\Users\Admin\AppData\Local\Temp\ipykernel_20352\4070490340.py:61: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  status = jumbotron_div.find(text='Trạng thái:').next.strip() if jumbotron_div.find(text='Trạng thái:') else 'N/A'


Time taken to fetch and process https://www.tratencongty.com/company/10ff363f4-cong-ty-tnhh-smart-bd/: 20.831860542297363 seconds
Time taken to fetch and process https://www.tratencongty.com/company/10ff363f4-cong-ty-tnhh-smart-bd/: 20.569395780563354 seconds
Time taken to fetch and process https://www.tratencongty.com/company/10ff363ed-cong-ty-tnhh-dau-tu-va-phat-trien-tan-hoa/: 20.870594024658203 seconds
Time taken to fetch and process https://www.tratencongty.com/company/10ff363ed-cong-ty-tnhh-dau-tu-va-phat-trien-tan-hoa/: 20.66215419769287 seconds
Time taken to fetch and process https://www.tratencongty.com/company/10ff363e6-cong-ty-tnhh-xe-may-xe-dien-phuong-nhi/: 20.54390811920166 seconds
Time taken to fetch and process https://www.tratencongty.com/company/10ff363e6-cong-ty-tnhh-xe-may-xe-dien-phuong-nhi/: 20.60209083557129 seconds
Time taken to fetch and process https://www.tratencongty.com/company/186f194a0-cong-ty-tnhh-quang-cao-va-noi-that-dung-nghia-media/: 20.6682298183441